In [18]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import FreqDist, pos_tag
import re

import json

In [19]:
with open ('./data/json_pr.json') as x:
    prj = json.load(x)
    
with open ('./data/json_ra.json') as y:
    raj = json.load(y)

In [20]:
df_prj = pd.DataFrame(prj)
df_raj = pd.DataFrame(raj)

In [21]:
# #confirm latest dfs: 0 nulls.  confirmed
# print(df_prj.isnull().sum())
# print(df_raj.isnull().sum())

# All posts left in original format, no special character elimination

# Structure analysis separate from diction:

Number of sentences, sentence length, and *rate* of punctuation use (. , ?) could be informative.  How preserve?  

2 separate analyses: nlp based on Bag of Words (0 structure or punctuation retention), and one based on structure.  

Word Frequencies (see lesson 5.04) counts *everything* - punctuation, numbers, etc.  But I'd rather use WordFreq, CountVec, etc for lexicon and reduce punctuation down to counts.

New columns for total wordcount, # sentences, average sentence length, total commas / ? / ! (which can be changed into per wordcount or per sentence 'rates' later).  

### Add column with sentence count
Methods based on experimentation in scratch notebook

In [22]:
df_prj['num_sentences'] = df_prj['selftext'].map(lambda x: len(re.split(r'[.!?]+', x))-1)
df_raj['num_sentences'] = df_raj['selftext'].map(lambda x: len(re.split(r'[.!?]+', x))-1)

### Add column with average sentence length (by word)

In [23]:
# len counts spaces and characters.  you want wordcount.
length = 'this is twenty lettersS..!?,'
len(length)

28

In [24]:
#df_raj_mini['words_per_sentence'] = 

def avg_words_per_sentence(df):
    
    all_sentences_bypost = []

    for i in df['selftext']:
        all_sentences_bypost.append(re.split(r'[.!?]+', i))
    
    avgwordsper = []

    for post in all_sentences_bypost:
        wordspersentence = []
        for i in post:
            wordspersentence.append(len(i.split()))
        avgwordsper.append(int(np.mean(wordspersentence).round()))
        
    return avgwordsper

In [25]:
df_raj['avg_words_per_sent'] = avg_words_per_sentence(df_raj)
df_prj['avg_words_per_sent'] = avg_words_per_sentence(df_prj)

### Add columns with total wordcount, commas, ?s, and !s

In [26]:
df_raj['wordcount'] = df_raj['selftext'].map(lambda x: len(x.split()))
df_prj['wordcount'] = df_prj['selftext'].map(lambda x: len(x.split()))

In [27]:
df_raj['comma_count'] = df_raj['selftext'].map(lambda x: x.count(','))
df_raj['qmark_count'] = df_raj['selftext'].map(lambda x: x.count('?'))
df_raj['exclamatios'] = df_raj['selftext'].map(lambda x: x.count('!'))

In [28]:
def count_punc(punc, df):
    perpost = []
    for i in df['selftext']:
        perpost.append(i.count(punc))
    return perpost

In [29]:
df_prj['comma_count'] = count_punc(',', df_prj)
df_prj['qmark_count'] = count_punc('?', df_prj)
df_prj['exclamatios'] = count_punc('!', df_prj)

In [30]:
## No. Use total comma count, ! count, ? count
## Can turn into 'rate' later.  Prob by sentence count, not wordcount...

# def punc_rate(punctuation, df):
    
#     puncs=[]
#     punc_rate=[]
    
#     for i in df['selftext']:
#         commas.append(len(re.findall(punctuation, i)))
    
#     for j in commas:
#         comma_rate.append(commas[j]/df['wordcount'][j])
        
#     return punc_rate

In [31]:
# df_raj['comma_rate'] = punc_rate(',', df_raj)
# df_prj['comma_rate'] = punc_rate(',', df_prj)

In [32]:
#overly complicated just use map + str.count
# run into issues with 0s with re.findall I think

# def punc_count(punc, df):
#     count = []
#     for i in df['selftext']:
#         count.append(len(re.findall(punc, i)))
#     return count

In [33]:
df_prj

,selftext,subreddit,created_utc,is_self,score,title,author,num_comments,timestamp,year,num_sentences,avg_words_per_sent,wordcount,comma_count,qmark_count,exclamatios
0,\n\nI remember during my first year of colleg...,pettyrevenge,1617115680,True,1,How the tables have turned.,BestGuessGuest,79,2021-03-30,2021,12,17,227,6,0,0
1,A bit long TDRL at the end. First post on mobi...,pettyrevenge,1617119446,True,1,I know how to use the radio,Negative_Shake1478,23,2021-03-30,2021,25,15,391,9,0,0
2,Senior year of high school. Last school dance ...,pettyrevenge,1617138131,True,1,Made a girl cry after she stuck gum in my hair,The_Bee_Sneeze,44,2021-03-30,2021,39,10,410,27,2,0
3,"When I was working as a barista, there were tw...",pettyrevenge,1617168650,True,1,Lady waits to use the bathroom every night at ...,snailtearstains,557,2021-03-30,2021,44,12,517,18,3,2
4,First let me say: I fucking hate it when peopl...,pettyrevenge,1617204366,True,1,Rude lady talking on phone,Xxtratourettestriall,20,2021-03-31,2021,8,26,233,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5908,I hate cleaning the shower. I think my wife g...,pettyrevenge,1468803783,True,45,Cleaning the shower,d0ntblink,12,2016-07-17,2016,3,16,65,3,0,0
5909,"Recently, I attended a sewing summer camp. Pre...",pettyrevenge,1468804377,True,2534,"Save 20 seconds, lose 5 minutes, b----.",Falcon10301,148,2016-07-17,2016,29,18,522,29,3,1
5910,Was at a show with my wife recently. We were ...,pettyrevenge,1468812440,True,154,"Be rude and pushy at a crowded show, enjoy a w...",Doodieboi7,8,2016-07-17,2016,8,15,134,2,0,0
5911,"So one night, a few months ago, a friend of mi...",pettyrevenge,1468827535,True,109,"Yeah, I got a tip for you....",AceBobcat,10,2016-07-17,2016,12,14,174,15,0,0


### Add column is_mobile to distinguish if posted by mobile user
Users generally announce '(on) Mobile" if they are posting from mobile phones, but there is no consistent format to this announcement.  Sometimes it's a sentence, sometimes a single word declaration.  ID'ing is_mobile by simply searching for the word 'mobile' will also flag posts containing words like 'automobile', 'immobile', 'mobile bank account', etc ('shitmobile' is one I found that I did not expect...).  
Spot check 100 posts from each df to get rate of false positives to see if an is_mobile column is informative.

---

Results from check: 
* relationship_advice had 10.6% false positives (10 of total 94)
* pettyrev had 8% false positive rate (8 of 100 random checks, total count 395)  

##### Assume a fp rate of 9%, still worth identifying.  Pretty consistent fp rate across both dataframes makes it a reliable enough metric.  Concatenate before adding column. 

In [47]:
#use ctrl-f in jq pretty-print to check if posting from mobile, or talking about mobile homes, etc. 
df_raj.loc[df_raj['selftext'].str.lower().str.contains('mobile')].to_json('./scratch/is_mobile_ra.json', orient='records')

In [48]:
# 10/94 records containing 'mobile' NOT posted from mobile user 
#is_mobile = false positive rate 10.6% for RA
df_raj.loc[df_raj['selftext'].str.lower().str.contains('mobile')].shape

(94, 16)

In [52]:
# repeat for pettyrev.  len = 395.  check first 100.
df_prj.loc[df_prj['selftext'].str.lower().str.contains('mobile')].head(100).to_json('./scratch/is_mobile_pr.json', orient='records')

In [62]:
combo_structured = pd.concat([df_prj, df_raj], axis = 0)

In [76]:
combo_structured['is_mobile'] = combo_structured['selftext'].apply(lambda x: 1 if 'mobile' in x.lower() else 0)

In [81]:
combo_structured.is_mobile.value_counts()

0    11519
1      489
Name: is_mobile, dtype: int64

In [83]:
#export csv and json versions
combo_structured.to_csv('./data/allposts_struc.csv', index=False)
combo_structured.to_json('./data/j_allposts_struc.json', orient='records')

## Special characters to scrub:
* characters with alphanum attached: I think remove these whole units first (or the attached text), otherwise left with 'n', 'amp', 'u username', etc that won't be picked up by subsequent alphanum filter
    * ' & amp ;
    * \n
    * u/username
    * (Mobile)
        * edit: leave mobile in place, could be differentiating feature between subreddits
* all non-alpha-numerics: \, /, &, * , "", '', -
* emojis
* any URLs:  start with https://...  
* I don't think numbers will be informative
* leave punctuation in place?  ADHD-comma anecdote, prevalence of ! ? might be informative?
    * RA likely contains ? all the time.

In [49]:
df_prj.loc[df_prj['selftext'].str.contains('mobile|Mobile')].shape

(393, 10)

In [50]:
df_raj.loc[df_raj['selftext'].str.contains('mobile|Mobile')].shape

(94, 10)

In [83]:
testpost.split()

['Sorry',
 'in',
 'advance',
 'if',
 'my',
 'texts',
 'are',
 'a',
 'bit',
 'sloppy..',
 "It's",
 'taken',
 'me',
 'a',
 'long',
 'while',
 'to',
 'finally',
 'write',
 'this',
 'out',
 'as',
 "I'm",
 'just',
 'very',
 'unsure.',
 'Side',
 'notes:',
 "I'm",
 'not',
 'from',
 'USA.',
 "I'm",
 'from',
 'Asia',
 '&amp;',
 'My',
 'country',
 'to',
 'hers',
 'is',
 'about',
 '30m',
 'flight',
 '&amp;',
 'cheap.',
 'Backstory;',
 'When',
 'I',
 'was',
 '14,',
 'I',
 'had',
 'my',
 'first',
 'ever',
 "'real'",
 'relationship.',
 'At',
 'the',
 'time',
 'it',
 'was',
 'solely',
 'online',
 'until',
 'I',
 'was',
 'able',
 'to',
 'fly',
 'over',
 'to',
 'her',
 'country',
 '(accompanied',
 'by',
 'my',
 "mom's",
 'friend)',
 'twice',
 'a',
 'month',
 'when',
 'I',
 'was',
 '15.',
 'Before',
 'you',
 'hammer',
 'about',
 'my',
 'parents',
 'not',
 'seeing',
 'the',
 'wrong',
 'in',
 'our',
 'age',
 'gap,',
 'she',
 'always',
 'made',
 'me',
 'tell',
 'them',
 'that',
 'we',
 'were',
 'only',
 'a